In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import glob
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.manifold import TSNE
from alexnet import AlexNet
from lenet5 import LeNet5
from data_loading import DataLoaderCreator

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def load_results(csv_path):
    return pd.read_csv(csv_path)

In [4]:
def load_model(model, model_path, dropout_rate=0, init_type='random'):
    if model == 'alexnet':
        model = AlexNet(dropout_rate=dropout_rate, init_type=init_type)
    elif model == 'lenet5':
        model = LeNet5(dropout_rate=dropout_rate, init_type=init_type)
    else:
        raise ValueError("Invalid model name")
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))  # CPU/GPU
    model.eval()
    return model

In [5]:
def evaluate_model(model, test_loader, device):
    model.to(device)
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = 100 * correct / total
    return accuracy

In [13]:
model_paths = glob.glob("saved_models/alexnet_dropout_seed_*.pth")
accuracies = []

loader_creator = DataLoaderCreator(seed=123)
test_loader = loader_creator.create_data_loader('cinic-10/test', batch_size=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for model_path in model_paths:
    print(f"Evaluating {model_path}...")
    model = load_model("alexnet", model_path, dropout_rate=0.2, init_type='random')
    accuracy = evaluate_model(model, test_loader, device)
    accuracies.append(accuracy)
    print(f"Accuracy: {accuracy:.2f}%")

mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies, ddof=1)

print(f"\nMean accuracy: {mean_accuracy:.2f}%")
print(f"Standard Deviation: {std_accuracy:.2f}%")

Evaluating saved_models\alexnet_dropout_seed_123.pth...
Accuracy: 61.06%
Evaluating saved_models\alexnet_dropout_seed_33.pth...
Accuracy: 61.17%
Evaluating saved_models\alexnet_dropout_seed_40.pth...
Accuracy: 59.54%

Mean accuracy: 60.59%
Standard Deviation: 0.91%
